<a href="https://colab.research.google.com/github/suu3/Capstone/blob/main/%5B2021_2_ML%5D_HW2_2_skeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 2021-2 Machine Learning (01)
## Homework #2: Support Vectore Machines Step 2 - Kernel Trick
---
Copyright (c) Code Designed by Prof. Jaehyeong Sim 

Department of Computer Science and Engineering

Ewha Womans University



## Read the csv file and convert the data into Numpy array

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

data = np.recfromcsv("haberman.csv", names=None)

x = data[:,0:3]
t = data[:,3]

## Visualize the data in 3d space

In [2]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame(data, columns=["Age", "Year", "PAD", "Class"])
fig = px.scatter_3d(df, x=df.Age, y=df.Year, z=df.PAD, color=df.Class, symbol=df.Class)
fig.update_traces(marker_size=2)
fig.update_traces(marker_color=1)
fig.update_coloraxes(showscale=False)
fig.update_layout(legend_itemsizing="constant")
fig.show()

## Kernel function

In [8]:
def kernel(x, y, sigma=3.0):
    # TODO: your job - use Gaussian as a kernel function:
    return np.exp(-np.linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

## Run convex opimization using cvxopt library

In [9]:
X = x
y = np.where(t==1, -1, 1)
m,n = X.shape
y = y.reshape(-1,1) * 1.

# TODO: your job - calculate the variable K
K = np.zeros((m, m))
for i in range(m):
  for j in range(m):
    K[i,j] = kernel(x[i], x[j])

P = cvxopt_matrix(np.outer(y,y) * K)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(-np.eye(m))
h = cvxopt_matrix(np.zeros(m))
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

cvxopt_solvers.options['show_progress'] = False

sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.ravel(sol['x'])

## Computing bias

In [10]:
sv = alphas > 1e-4
a = alphas[sv]
sv_y = y[sv].ravel()
ind = np.arange(len(alphas))[sv.ravel()]
b = 0
for n in range(len(a)):
  b += sv_y[n]
  b -= np.sum(a * sv_y * K[ind[n], sv])
b /= len(a)

## Project function

In [11]:
def project(X, a_, sv_y_, sv_, b_):
  y_predict = np.zeros(len(X))
  for i in range(len(X)):
    s = 0
    for a_i, sv_y_i, sv_i in zip(a_, sv_y_, sv_):
      s += a_i * sv_y_i * kernel(X[i], sv_i)
    y_predict[i] = s
  return y_predict + b_

## Computing final training accuracy

In [24]:
y_ = project(x, a, sv_y, X[sv], b)

# TODO: your job - calculate the final training accuracy
count = 0
output = np.where(y_ >=0, 2, 1)
for i in range(t.size):
  if(t[i]==output[i]):
    count = count + 1
accuracy = count / t.size
print(accuracy)

0.8529411764705882


## Visualize the decision surface

In [13]:
import plotly.graph_objects as go

x_grid = np.linspace(30, 80, 10)
y_grid = np.linspace(58, 70, 10)
z_grid = np.linspace(0, 60, 10)
x_mgrid, y_mgrid, z_mgrid = np.meshgrid(x_grid, y_grid, z_grid)
x_grid2 = np.array([[xi, yi, zi] for xi, yi, zi in zip(x_mgrid.ravel(), y_mgrid.ravel(), z_mgrid.ravel())])
p_grid = project(x_grid2, a, sv_y, X[sv], b)
fig.add_trace(go.Isosurface(x=x_mgrid.ravel(), y=y_mgrid.ravel(), z=z_mgrid.ravel(), value=p_grid, isomin=-0.001, isomax=0.001, opacity = 0.5, showscale=False))
fig.show()

### ❔ What is the final accuracy of your trained model? 
### 🗒 Answer here: 0.8529411764705882

### ❔ Compare your accuracy with the accuracy result of the linear SVM in Step1,  explain why you get such result. 
### 🗒 Answer here: step1에서 약 0.26이었던 정확도가 약 0.85로 확연하게 올랐다. 선형으로 분류할 수 없었던 벡터들의 클래스들을 가우시안 커널로 선형 분류 가능하게 바꿨기 때문이다.